In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from math import ceil, log, exp
from sklearn import preprocessing
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.colors as mcolors


tf.keras.utils.set_random_seed(seed=100)

2024-03-29 09:38:21.665383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## AutoEncoder 

In [2]:
class AutoEncoder(tf.keras.Model):
    def __init__(self, x_train):
        super().__init__()
        self.encoder = self.encoder_model()
        self.decoder = self.decoder_model()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.train_data = x_train
        

    def encoder_model(self):
        encoder_inputs = keras.Input(shape=(11,)) # the shape is the number of dimensions and the length of data that will be put in. 
        # i.e. 6dims and 1 data points
        
        

        encoded = layers.Dense(9, activation='tanh')(encoder_inputs)
        encoded = layers.Dense(7, activation='tanh')(encoded)
        encoded = layers.Dense(5, activation='tanh')(encoded)
        latent_space = layers.Dense(3, activation='tanh')(encoded) # this is the latent space (the output of the encoder)
        
        encoder = keras.Model(encoder_inputs, latent_space) # this defines the model in the keras computational graph
        encoder.compile(optimizer='Adam', loss='mse') # the compile step of the keras model
        encoder.summary()
        return encoder # returns the compiled model
    
    def decoder_model(self):
        decoder_inputs = keras.Input(shape=(3,)) # these are the inputs of the decoder (3dims (the latent space), 1 number of data points)
        decoded = layers.Dense(5, activation='tanh')(decoder_inputs)
        decoded = layers.Dense(7, activation='tanh')(decoded)
        decoded = layers.Dense(9, activation='tanh')(decoded)     
        output_space = layers.Dense(11, activation='sigmoid')(decoded) # the output (decoded latent space, should be the inputs)
        
        decoder = keras.Model(decoder_inputs, output_space) # defining the decoder as a Model
        decoder.compile(optimizer='Adam', loss='mse') # compiling the model
        decoder.summary()
        return decoder # returning the compiled model
    
    
    
    def train_step(self, data): # as we have two parts of the model, we have to define how the model should be trained to access the latent space
        with tf.GradientTape() as tape:
            latent_output = self.encoder(data)
            reconstructed_data = self.decoder(latent_output)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mean_squared_logarithmic_error(data, reconstructed_data)))
            total_loss = reconstruction_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        return {'total_loss': self.total_loss_tracker.result(),
                'reconstruction_loss': self.reconstruction_loss_tracker.result()}

## Preprocessing Functions

In [3]:
class preprocessing:

    def map_unique_values(self, arr):
        unique_values = list(set(arr))
        unique_values.sort() 
        value_to_number_mapping = {value: i + 1 for i, value in enumerate(unique_values)}
        mapped_array = np.array([value_to_number_mapping[value] for value in arr])
        return mapped_array
    
    def map_unique_values_to_columns(self, np_array):
        mapped_columns = []
        for col_idx in range(np_array.shape[1]):
            mapped_column = self.map_unique_values(np_array[:, col_idx])
            mapped_columns.append(mapped_column)
        return np.array(mapped_columns).T
    
    
    def min_max(array):
        min_val = np.min(array, axis=0)
        max_val = np.max(array, axis=0)
        normalized_array = (array - min_val) / (max_val - min_val)
        return normalized_array


    def min_max_odd_columns(array):
        normalized_array = np.empty_like(array)
        for i in range(array.shape[1]):
            if i % 2 != 0:  # Even columns, just copy
                normalized_array[:, i] = array[:, i]
            else:  # Odd columns, perform min-max normalization
                min_val = np.min(array[:, i])
                max_val = np.max(array[:, i])
                normalized_array[:, i] = (array[:, i] - min_val) / (max_val - min_val)
        return normalized_array



## K-means Clustering

In [4]:
class clustering:

    def assign_clusters_find_distances(self, new_data, centroids):
            # Calculate distances to centroids
            distances = np.linalg.norm(new_data[:, np.newaxis] - centroids, axis=2)
            
            # Assign each point to the closest centroid
            cluster_assignment = np.argmin(distances, axis=1)
            
            # Get distances to the closest centroid
            min_distances = np.min(distances, axis=1)
            
            return cluster_assignment, min_distances


    def kmean_clustering_with_distances(self, data, num_clusters):
        from sklearn.cluster import KMeans
        
    
        
        num_columns = data.shape[1]
        centroids_list = []
        assignment_distances_list = []
        
        # Iterate over each column
        for i in range(num_columns):
            # Remove the i-th column from the data
            data_subset = np.delete(data, i, axis=1)
            
            # Perform K-means clustering
            kmeans = kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=10).fit(data_subset)
            
            # Get cluster centroids
            centroids = kmeans.cluster_centers_
            centroids_list.append(centroids)
            
            # Assign clusters and distances
            cluster_assignment, distances = self.assign_clusters_find_distances(data_subset, centroids)
            assignment_distances_list.append(cluster_assignment)
            assignment_distances_list.append(distances)
        
        # Stack centroids into a single array
        stacked_centroids = np.stack(centroids_list, axis=1)
    
        # Stack assignments and distances into a single array
        stacked_assignment_distances = np.column_stack(assignment_distances_list)
        
        return stacked_centroids, stacked_assignment_distances




    

    def kmeans_clustering_with_distances_new_data(self, stacked_centroids, data): 
            num_columns = stacked_centroids.shape[1]
            assignment_distances_list = []
            
            # Iterate over each column
            for i in range(num_columns):
                # Get the centroids for the current iteration
                centroids = stacked_centroids[:, i, :]
                
                # Remove the i-th column from the data
                data_subset = np.delete(data, i, axis=1)
                
                # Assign clusters and distances using the provided centroids
                cluster_assignment, distances = self.assign_clusters_find_distances(data_subset, centroids)
                assignment_distances_list.append(cluster_assignment)
                assignment_distances_list.append(distances)
            
            # Stack assignments and distances into a single array
            assignment_distances = np.column_stack(assignment_distances_list)
            
            return assignment_distances





## Data Analysis Functions

In [5]:
class data_analysis:

    def calculate_gradient(self, x, y):
        if len(x) != len(y):
            raise ValueError("Arrays must be of the same length")
        
        # Perform linear regression to find the slope (gradient)
        m, _ = np.polyfit(x, y, 1)
        
        return m

    def calculate_average_grad(self, data_set1, data_set2):
        gradients = []
        for i in range(data_set1.shape[1]):
            gradient = self.calculate_gradient(data_set1[:, i], data_set2[:, i])
            gradients.append(gradient)
        
        gradients = np.array(gradients)
        average_gradient = np.mean(gradients)
        
        return gradients, average_gradient


    def calculate_r_square(self, array1, array2):
            if array1.shape[1] != array2.shape[1]:
                raise ValueError("Arrays must have the same number of columns")
            
            r_squared_values = []
            for i in range(array1.shape[1]):
                correlation_matrix = np.corrcoef(array1[:, i], array2[:, i])
                correlation_coefficient = correlation_matrix[0, 1]
                r_squared = correlation_coefficient ** 2
                r_squared_values.append(r_squared)
        
            r_squared_values = np.array(r_squared_values)
            average_r_squared = np.mean(r_squared_values)
            
            return r_squared_values, average_r_squared
    
    def plot_hexbin_log_frequency(x, y, xlabel="", ylabel="", title="", gridsize=100, cmap='viridis', colorbar=True):
        """
        Function to create a hexbin plot with log-scaled frequency using Matplotlib.
        """
        plt.figure(figsize=(8, 6))
        
        # Calculate the hexbin frequencies
        hb = plt.hexbin(x, y, gridsize=gridsize, cmap=cmap)
        counts = hb.get_array()
        nonzero_counts = counts[counts != 0]
        
        # Set the norm to LogNorm with vmin=0
        norm = mcolors.LogNorm(vmin=1, vmax=nonzero_counts.max())
        hb = plt.hexbin(x, y, gridsize=gridsize, cmap=cmap, norm=norm)
        
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        
        if colorbar:
            plt.colorbar(hb, label='log(count)')
        
        plt.show()

## Preprocessing 

In [6]:
x_data_raw = np.loadtxt('train4 .csv',delimiter = ',') #import of data as numpy array

In [7]:
#seperate categorical and continuous

categorical_data = np.delete(x_data_raw, 5, axis=1)
x6 = x_data_raw[:, 5]



In [8]:
#create instances for classes
preprocessor = preprocessing()
Cluster = clustering()
Analyse = data_analysis()

In [9]:
#Preprocess Categorical Data

#apply even spacing

mapped = preprocessor.map_unique_values_to_columns(categorical_data)

#normalise 
mapped_min_max = preprocessing.min_max(mapped)

#cluster
centroids, clustered = Cluster.kmean_clustering_with_distances(mapped_min_max, 10)

#add min max to cluster number
normalised_clustered = preprocessing.min_max_odd_columns(clustered)

print(normalised_clustered)





[[0.44444444 0.089851   0.88888889 ... 0.10202549 0.66666667 0.08914823]
 [0.11111111 0.25710347 0.         ... 0.23100372 0.55555556 0.23470316]
 [0.44444444 0.04883137 0.88888889 ... 0.15936334 0.66666667 0.14677303]
 ...
 [0.33333333 0.21887122 0.77777778 ... 0.2127822  0.11111111 0.21588971]
 [0.11111111 0.17342957 0.33333333 ... 0.21553291 0.66666667 0.23521337]
 [0.44444444 0.05476121 0.88888889 ... 0.07720942 0.66666667 0.07693073]]


In [10]:
#Preprocess Continuous Data

log_x6 = np.log10(x6)
x6_logged_normalized = preprocessing.min_max(log_x6)


In [11]:
Input_data = np.column_stack((normalised_clustered, x6_logged_normalized))

In [12]:
df = pd.DataFrame(clustered, columns=['1', '2', '3','4','5','6','7','8','9','10'])
df.to_csv('Clustered input') 

In [13]:
x_train = Input_data

## Train AutoEncoder

In [14]:

AutoEncoder = AutoEncoder(x_train)
AutoEncoder.compile(optimizer=keras.optimizers.Adam())
AutoEncoder.fit(x_train, epochs= 2000 , batch_size=200)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense (Dense)               (None, 9)                 108       
                                                                 
 dense_1 (Dense)             (None, 7)                 70        
                                                                 
 dense_2 (Dense)             (None, 5)                 40        
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 236 (944.00 Byte)
Trainable params: 236 (944.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model_1"
______

## Analyse Training 

In [ ]:
#Run with Trianing Data

latent_space_representation = AutoEncoder.encoder.predict(Input_data)

Output_data = AutoEncoder.decoder.predict(latent_space_representation)

In [ ]:
#Find Reconstruction Loss
Reconstruction_loss = np.mean(np.square(x_train - Output_data), axis = 1)
Threshold = np.percentile(Reconstruction_loss, 99.0)
print(Threshold)


In [ ]:
#Find Gradients
all_gradients, average_gradient = Analyse.calculate_average_grad(Input_data, Output_data)
print(average_gradient)


In [ ]:
#Find r^2 values
all_r_squared, average_r_squared = Analyse.calculate_r_square(Input_data, Output_data)
print(average_r_squared)

In [ ]:
#Plot input vs output
data_analysis.plot_hexbin_log_frequency(Input_data[:,10], Output_data[:, 10] , xlabel='Input', ylabel='Output', title='')

# Test Trained Model

In [ ]:
#import test data as numpy array 
Testing_data = np.loadtxt('testing_data2.csv',delimiter = ',')

## Preprocess Test data

In [ ]:
#seperate categorical and continuous

categorical_data_test = np.delete(Testing_data, 5, axis=1)
x6_test = Testing_data[:, 5]

In [ ]:
#Preprocess Categorical Data

#apply even spacing
mapped_test = preprocessor.map_unique_values_to_columns(categorical_data_test)

#normalise 
mapped_min_max_test = preprocessing.min_max(mapped_test)

#cluster with same centroids used for training data 
clustered_test = Cluster.kmeans_clustering_with_distances_new_data(centroids, mapped_min_max_test) 

#add min max to cluster number
normalised_clustered_test = preprocessing.min_max_odd_columns(clustered_test)

print(normalised_clustered_test)


In [ ]:
#Preprocess continuous data

log_x6_test = np.log10(x6_test)
x6_logged_normalized_test = preprocessing.min_max(log_x6_test)



In [ ]:
Input_data_test = np.column_stack((normalised_clustered_test, x6_logged_normalized_test))

## AutoEncoder on Test Data

In [ ]:
latent_space_representation_test = AutoEncoder.encoder.predict(Input_data_test)

Output_data_test = AutoEncoder.decoder.predict(latent_space_representation_test)

## Find Anomalies using Training Data Threshold

In [ ]:
Reconstruction_loss = np.mean(np.square(Input_data_test - Output_data_test), axis = 1)

In [ ]:
anomaly_truth = Reconstruction_loss > Threshold

anomaly_indices = np.where(anomaly_truth)[0]

anomalies = Input_data_test[anomaly_truth]

print("Anomalies:")
print(anomalies)

print("Indices of anomalies in Test_data:")
print(anomaly_indices)

print("Number of anomalies:", len(anomalies))

## Test Data Analysis

In [ ]:
Test_grad, Test_grad_average = Analyse.calculate_average_grad(Input_data_test, Output_data_test)
print(Test_grad_average)

In [ ]:
Test_R2, Test_R2_average = Analyse.calculate_r_square(Input_data_test, Output_data_test)
print(Test_R2_average)

## Compare to George IDs

In [ ]:
df = pd.read_csv('George_Anomalies.csv')
George_IDs = df['id']
array_of_values = np.array(George_IDs, dtype=int)
print(array_of_values)

In [ ]:
# Calculate the intersection of the two arrays
intersection = np.intersect1d(George_IDs, anomaly_indices)

# Get the number of values shared
shared_count = len(intersection)

print("Number of shared values:", shared_count)